# GridSearchTest

In [4]:
import sys, os
import numpy as np


def importOwnLib():
    if '/home/miruware/aProjects/lib' not in sys.path:
        sys.path.append('/home/miruware/aProjects/lib')
        print("lib path is successfully appended.")
    else:
        print("lib path is already exists.")

importOwnLib()
import sonyalib as sonya

import importlib
importlib.reload(sonya)

import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout
from keras.backend.tensorflow_backend import set_session
from keras.losses import binary_crossentropy
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import Adam

from sklearn.model_selection import train_test_split, GridSearchCV


# ==================================================



config = tf.ConfigProto()
config.gpu_options.visible_device_list = "0"
set_session(tf.Session(config=config))


def create_model(num_input=1, dropout=0, learning_rate=0.005, neurons1=9, neurons2=9):
    global num_features
    keras.backend.clear_session()

    ## 모델 구성하기
    model = Sequential()

    model.add(Dense(neurons1, activation='relu', input_dim=num_features, kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Dense(neurons2, activation='relu', kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Dropout(dropout))
    model.add(Dense(1, activation='sigmoid', kernel_initializer='he_normal'))

    ## 모델 컴파일
    model.compile(optimizer=Adam(learning_rate), loss=binary_crossentropy, metrics=['accuracy'])

    # model.summary()

    return model


lib path is already exists.


In [5]:
## >>>>>>>>>>>>>>>>>> Model Tuning ( Grid Search ) <<<<<<<<<<<<<<<<<<<<
# fit random seed for reproductivity
seed = 7
np.random.seed(seed)

# Load metadata
file_path0 = './BRC_input_201116.xlsx'
metadata = sonya.get_original_metadata(file_path0)

# properties = list(metadata.columns.values)
# properties.remove('label')
# X = metadata[properties]
# y = metadata['label']

X = metadata[[
    'sex',
    'age',
    'LR',
    'cT',
    'cN',
    'cAverage',
    # 'cSD',
    # 'aAverage',
    # 'aSD',
    # 'lMax',
    # 'homogeneous',
    # 'hetero',
    # 'rim',
    # 'clustered',
    # 'non-mass',
    # 'AorCa',
    'LymAo',
    # 'LymCa'
]]

y = metadata['label']

num_features = len(X.columns)

# seperate dataset to train data & test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)


# create model
# model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=batch_size)
model = KerasClassifier(build_fn=create_model)

batch = [10, 20]
epochs = [10, 50, 100]
lr = [0.001,  0.05]
neurons1 = [3,  11]
neurons2 = [3,  11]

grid = dict(
    neurons1=neurons1,
    neurons2=neurons2,
    batch_size=batch,
    learning_rate=lr
)

grid = GridSearchCV(estimator=model, param_grid=grid, cv=10)
grid_result = grid.fit(X_train, y_train) # 최적의 hyperparameters를 찾는데 testset을 사용하지 않았다.

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Instructions for updating:
Use tf.cast instead.
Epoch 1/1
176/176 [==============================] - 0s 187us/step
Epoch 1/1
176/176 [==============================] - 0s 184us/step
Epoch 1/1
176/176 [==============================] - 0s 180us/step
Epoch 1/1
176/176 [==============================] - 0s 179us/step
Epoch 1/1
176/176 [==============================] - 0s 181us/step
Epoch 1/1
176/176 [==============================] - 0s 175us/step
Epoch 1/1
177/177 [==============================] - 0s 177us/step
Epoch 1/1
177/177 [==============================] - 0s 177us/step
Epoch 1/1
177/177 [==============================] - 0s 175us/step
Epoch 1/1
177/177 [==============================] - 0s 184us/step
Epoch 1/1
176/176 [==============================] - 0s 179us/step
Epoch 1/1
176/176 [==============================] - 0s 192us/step
Epoch 1/1
176/176 [==============================] - 0s 175us/step
Epoch 1/1
176/176 [==============================] - 0s 172us/step
Epoch 1/1
176/

In [6]:
grid_result.best_params_

{'batch_size': 10, 'learning_rate': 0.05, 'neurons1': 11, 'neurons2': 3}

In [8]:
grid_result.best_params_['batch_size']

10

In [9]:
grid_result.score(X_test, y_test)

84/84 [==============================] - 0s 1ms/step


0.7142857128665561

In [11]:
test_accuracy = 0.7142857128665561

In [22]:
properties = list(metadata.columns.values)
print(properties)

['sex', 'age', 'LR', 'cT', 'cN', 'cAverage', 'cSD', 'aAverage', 'aSD', 'lMax', 'homogeneous', 'hetero', 'rim', 'clustered', 'non-mass', 'AorCa', 'LymAo', 'LymCa', 'label']


In [25]:
test = properties
test.remove('sex')

ValueError: list.remove(x): x not in list

In [24]:
print(test)
print(properties)

None
['age', 'LR', 'cT', 'cN', 'cAverage', 'cSD', 'aAverage', 'aSD', 'lMax', 'homogeneous', 'hetero', 'rim', 'clustered', 'non-mass', 'AorCa', 'LymAo', 'LymCa', 'label']


In [14]:
list(X_test.columns.values), grid_result.best_params_, test_accuracy

(['sex', 'age', 'LR', 'cT', 'cN', 'cAverage', 'LymAo'],
 {'batch_size': 10, 'learning_rate': 0.05, 'neurons1': 11, 'neurons2': 3},
 0.7142857128665561)

In [21]:
with open("./normalized_meta.txt",'a') as f:
    data = str(', '.join(list(X_test.columns.values)) + '\n' + str(grid_result.best_params_) +'\n' +  str(test_accuracy) + '\n')
    f.write(data)

In [1]:
def get_lr_metric(optimizer):
    def lr(y_true, y_pred):
        return optimizer.lr
    return lr
def mlp_model(num_input, dropout=0, lr=0.005, l1=9, l2=9):
    keras.backend.clear_session()
    ## 모델 구성하기
    model = Sequential()
    # print learning rate
    optimizer = adam(lr=0.001)
    lr_metric = get_lr_metric(optimizer)
    
    model.add(Dense(l1, activation='relu', input_dim=num_input, kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Dense(l2, activation='relu', kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Dropout(dropout))
    model.add(Dense(1, activation='sigmoid', kernel_initializer='he_normal'))

    ## 모델 컴파일
    model.compile(optimizer=adam(lr), loss=losses.binary_crossentropy, metrics=['accuracy', lr_metric])

    # model.summary()
    return model